In [2]:
# import libs
import os
import numpy as np 
import torch 
import glob 
import torch.nn as nn 
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam 
from torch.autograd import Variable
import torchvision 
import pathlib

/home/anirudh/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# if cuda available 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
print(device)

cpu


In [5]:
# Transforms
transformer = transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.5 , 0.5 , 0.5 ],[0.5 , 0.5 , 0.5 ]),
])

In [6]:
# set directory paths 
base_dir = "basedata"
training = "training"
validationString = "validation"

# training ,validation locations 
training_path = os.path.join(base_dir,training)
test_path = os.path.join(base_dir,"testing")

In [80]:
# DataLoader 
train_loader = DataLoader(
    torchvision.datasets.ImageFolder(training_path , transform = transformer),
    batch_size = 12 , shuffle= True
)
test_loader = DataLoader(
    torchvision.datasets.ImageFolder(test_path , transform = transformer),
    batch_size = 12 , shuffle= True
)

In [81]:
# get class
classes = os.listdir(training_path)

In [113]:
# define the Neural Net 
class ConvNet(nn.Module):
    def __init__(self,num_classes = 41):
        super(ConvNet,self).__init__()

        self.conv1 = nn.Conv2d(in_channels= 3 , out_channels=12 , kernel_size= 3, stride = 1 , padding = 1)
         
        self.bn1 = nn.BatchNorm2d(num_features = 12)

        self.relu1 = nn.ReLU()

        self.pool = nn.MaxPool2d(kernel_size= 2)
        # 
        self.conv2 = nn.Conv2d(in_channels = 12 , out_channels=20 , kernel_size= 3 , stride=1 , padding= 1)

        self.relu2 = nn.ReLU()

        self.conv3 = nn.Conv2d(in_channels= 20 , out_channels= 32 , kernel_size = 3 , stride = 1 , padding = 1)

        self.bn3 = nn.BatchNorm2d(num_features = 32)

        self.relu3 = nn.ReLU()

        self.fc = nn.Linear(in_features= 32*150*150 , out_features= num_classes)
    
    def forward(self,input):
        # print(input.shape)
        output = self.conv1(input)
        # print(output.shape)
        output = self.bn1(output)
        # print(output.shape)
        output = self.relu1(output)
        # print(output.shape)

        output = self.conv2(output)
        # print(output.shape)
        output = self.relu2(output)
        # print(output.shape)

        output = self.conv3(output)
        # print(output.shape)
        output = self.bn3(output)
        # print(output.shape)
        output = self.relu3(output)
        # print(output.shape)

        # resizing 
        output  = output.view(-1,32*150*150)
        # print(output.shape)
        output = self.fc(output)
        return output


In [114]:
model = ConvNet(num_classes=41).to(device)

In [115]:
# Optimizer and loss function 
optimizer = Adam(model.parameters(),lr = 0.001 , weight_decay=0.0001)
loss_function = nn.CrossEntropyLoss()


In [116]:
num_epochs = 10 

In [117]:
# calculate testing_training , size 
train_count = len(glob.glob(training_path+'/**/*.JPG'))
test_count = len(glob.glob(test_path+'/**/*.JPG'))

In [122]:
best_accuracy = 0.0 

for epoch in range(num_epochs):
    model.train()

    train_accuracy = 0.0
    train_loss = 0.0 
    # print('Epoch : ' + str(epoch)+'Train Loss : ' + str(int(train_loss))+ 'Train Accuracy : '+str(int(train_accuracy)))
    for i , (images,labels) in enumerate(train_loader):
        if(torch.cuda.is_available()):
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
        optimizer.zero_grad()

        outputs = model(images)
        loss = loss_function(outputs , labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.cpu().data*images.size(0)
        _ , prediction = torch.max(outputs.data , 1)

        train_accuracy += int(torch.sum(prediction == labels.data))

    train_accuracy  = train_accuracy/train_count

    model.eval()
    print('Epoch : ' + str(epoch)+'Train Loss : ' + str(int(train_loss))+ 'Train Accuracy : '+str(int(train_accuracy)))

    # test_accuracy = 0.0
    # for i , (images,labels) in enumerate(test_loader):
    #     if(torch.cuda.is_available()):
    #         images = Variable(images.cuda())
    #         labels = Variable(labels.cuda())

    #     outputs = model(images)
    #     loss = loss_function(outputs , labels)
    #     _ , prediction = torch.max(outputs.data , 1)

    #     test_accuracy += int(torch.sum(prediction == labels.data))
        
    # test_accuracy  = test_accuracy/train_count
    
    # print('Epoch : ' + str(epoch)+'Train Loss : ' + str(int(train_loss))+ 'Train Accuracy : '+str(int(train_accuracy)))

    # if test_accuracy > best_accuracy:
    #     torch.save(model.state_dict(),'best_checlpoint_model')
    #     best_accuracy = test_accuracy

Epoch : 0Train Loss : 3412Train Accuracy : 0
Epoch : 1Train Loss : 1349Train Accuracy : 0
Epoch : 2Train Loss : 1060Train Accuracy : 0
Epoch : 3Train Loss : 660Train Accuracy : 0


In [46]:
train_loader

In [95]:
import cv2
import matplotlib.pyplot as plt 

In [103]:

image_Fpath = os.path.join(training_path,os.listdir(training_path)[0])
img_path = os.path.join(image_Fpath,os.listdir(image_Fpath)[0])

In [104]:
from PIL import Image
img1 = Image.open(img_path)

In [109]:
# Transforms
transformer1 = transforms.Compose([
    transforms.Resize((150,150)),
    transforms.ToTensor()
])



In [110]:
img_tensor = transformer1(img1)

In [111]:
img_tensor.shape

torch.Size([3, 150, 150])

In [44]:
trial = torch.randn(256, 32, 75, 75)

In [45]:
trial.shape

torch.Size([256, 32, 75, 75])

In [46]:
new_trial  = trial.view(-1,32*75*75)
new_trial.shape

torch.Size([256, 180000])